In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
# import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
import xgboost as xgb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
f = open('nb2.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [4]:
f1 = open('xgb', 'rb')
XGBClassifier = pickle.load(f1)
f1.close()

In [5]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [6]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [7]:

def attributeselection(tweet,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove username
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = re.sub(r'\\s+',r' ', tweet, flags=re.I)
    tweet = re.sub("([^\x00-\x7F])+"," ",tweet) #remove all non-english characters
    tweet = re.sub(r"\s+"," ", tweet, flags = re.I) #remove multiple spaces
    text = tweet
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = XGBClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [8]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']

        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [9]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @jonrappoport: #Coronavirus: creating the illusion of a pandemic through diagnostic tests...

Nailed them, with their own words.

https:…

['rt', 'coronavirus', 'creating', 'illusion', 'pandemic', 'diagnostic', 'tests', '...', 'nailed', 'words', 'https', '…']

 Class: Not Relevant

Tweet: RT @ScottImmordino: ' ... the Constitution grants citizens habeas corpus... arrestees the right to appear in front of a judge and ask to be…

['rt', '...', 'constitution', 'grants', 'citizens', 'habeas', 'corpus', '...', 'arrestees', 'right', 'appear', 'front', 'judge', 'ask', 'be…']
[25, 5, 1, 0, 24] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[25 5 1 0 24]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: @Jack needs to replace @Twitter employees who practice unsafe censorship. Dogmas kill.

['needs', 'replace', 'employees', 'practice', 'unsafe', 'censorship', 'dogmas', 'kill']

 Class: Not Relevant

Tweet: Panic As Another Man Tests Positive For Coronavirus In Bauchi https://t.co/4KzsCiKDh6 https://t.co/hd8WaOhFL4

['panic', 'another', 'man', 'tests', 'positive', 'coronavirus', 'bauchi']

 Class: Not Relevant

Tweet: RT @GarciaRosalind: What is there to suspend.....???

['rt', 'suspend', '...', '..']
[12, 4, 1, 0, 11] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[12 4 1 0 11]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @4everNeverTrump: 2.5 weeks ago, @senatemajldr adjourned the Senate for a 4-day recess so he could travel back to Kentucky to party with…

['rt', '2.5', 'weeks', 'ago', 'adjourned', 'senate', '4-day', 'recess', 'could', 'travel', 'back', 'kentucky', 'party', 'with…']

 Class: Not Relevant

Tweet: RT @Shem_Infinite: Nice! We went from four days, to 48 hours, to an hour, to 15 minutes, to 5 minutes, and now we're at 2 minutes. 

At the…

['rt', 'nice', 'went', 'four', 'days', '48', 'hours', 'hour', '15', 'minutes', '5', 'minutes', "'re", '2', 'minutes', 'the…']
[36, 2, 1, 0, 35] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[36 2 1 0 35]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



...

Tweet: Can we help this pastor's family? They need  help with cost of funeral for their dad, a faithful pastor, who died o… https://t.co/8FUoeiBZTx

['help', 'pastor', "'s", 'family', 'need', 'help', 'cost', 'funeral', 'dad', 'faithful', 'pastor', 'died', 'o…']
[27, 4, 1, 0, 26] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[27 4 1 0 26]] (1, 5)
PRIORITY:  [0]

 Class: Relevant

Tweet: RT @neeratanden: Over 3100 people have died in the US from the coronavirus. More than died on 9/11. Only 158 in South Korea, though we both…

['rt', '3100', 'people', 'died', 'us', 'coronavirus', 'died', '9/11', '158', 'south', 'korea', 'though', 'both…']
[29, 3, 13, 12, 16] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[29 3 13 12 16]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @TommyG: 13/ we also recently have rumors that they are both dead and Trump almost tweeted it, on top of the rumors that people have sea…

['rt', '13/', 'also', 'recently', 'rumors', 'dead', 'trump', 'almost', 'tweeted', 'top', 'rumors', 'people', 'sea…']
[28, 4, 1, 0, 27] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[28 4 1 0 27]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


KeyboardInterrupt: 